In [ ]:
import pylab as plt
import tqdm as tqdm
import pandas as pd
from herbie import FastHerbie
import cartopy.crs as ccrs
import numpy as np
import xarray as xr

# Set date ranges for simulation 

In [ ]:
#DATES = pd.date_range('2021-07-10 00:00', '2021-07-11 00:00', freq='1H')
#DATES = pd.date_range('2021-07-10 00:00', '2021-07-11 00:00', freq='1H')
DATES = pd.date_range('2021-03-14 00:00', '2021-03-15 00:00', freq='1H')
#DATES = pd.date_range('2021-01-29 00:00', '2021-01-31 00:00', freq='1H')
#DATES = pd.date_range('2021-01-29 12:00', '2021-01-30 00:00', freq='1H')
width = 24              # Halfwidth of data subset
lon_center = -124.833  # Longitude center point
lat_center = 41.13476  # Latitude center point

buoy_lat = 40.964533
buoy_lon = -124.593883

#lon_center = -70.5748
#lat_center = 40.1760

out_file = 'test_out_frontal.nc'

# Set-up map projection for HRRRR

In [ ]:
projection = ccrs.LambertConformal(central_longitude=-97.5,
                             central_latitude=38.5,
                             standard_parallels=[38.5],
                             globe=ccrs.Globe(ellipse="sphere"))

# Set-up map projection for HRRRR

In [ ]:
projection_pinacles = ccrs.LambertConformal(central_longitude= lon_center,
                             central_latitude=lat_center,
                             standard_parallels=[27.0,47.0],
                             globe=ccrs.Globe(ellipse="sphere"))

# Instantiate Herbie Classes

In [ ]:
H = FastHerbie(DATES, model="hrrr", product="sfc", save_dir='/Users/pres026/Research/Datasets/hrrr_data/hrrr')
Hnat = FastHerbie(DATES, model="hrrr", product="nat", save_dir='/Users/pres026/Research/Datasets/hrrr_data/hrrr')

# Download Surface Data

In [ ]:
surface_name = {} 
surface_name['VIL'] = "VIL"
surface_name['SST'] = ":TMP:surface"  
surface_name['T2m'] ="TMP:2 m"
surface_name['QV2m'] ="SPFH:2 m"
surface_name['U80'] = "UGRD:80 m"
surface_name['V80'] = "VGRD:80 m"
surface_name['U10'] = "UGRD:10 m"
surface_name['V10'] = "VGRD:10 m"
surface_name['PSFC'] = "PRES:surface"
surface_name['LHTFL'] = "LHTFL:surface"
surface_name['SHTFL'] = "SHTFL:surface"
surface_name['DSWRF'] = "DSWRF:surface"
surface_name['LCDC'] = "LCDC"
surface_name['WIND'] = "WIND"
surface_name['HPBL'] = "HPBL"
surface_name['LAND'] = "LAND"
surface_name['DLWRF'] = "DLWRF:surface"
for name, regex in tqdm.tqdm(surface_name.items()):
    H.download(regex)
    

In [ ]:
t2m = H.xarray(surface_name['T2m'])

# Download Pressure level Data

In [ ]:
level_name = {} 
level_name["U"] = ":(?:U)GRD:[0-9]+ hybrid"
level_name["V"] = ":(?:V)GRD:[0-9]+ hybrid"
level_name["P"] = ":(?:PRES):[0-9]+ hybrid"
level_name["T"] = ":(?:TMP):[0-9]+ hybrid"
level_name["QV"] = ":(?:SPFH):[0-9]+ hybrid"
level_name["QC"] = ":(?:CLMR):[0-9]+ hybrid"
level_name["QI"] = ":(?:CIMIXR):[0-9]+ hybrid"
level_name["Z"] =":(?:HGT:)[0-9]+ hybrid"

for name, regex in tqdm.tqdm(level_name.items()):
    Hnat.download(regex)

# Define a function for finding the closest gridpoint to the data subset center

In [ ]:
def get_nearest_point(projection, longitude, latitude):
    xa = np.arange(-2.698e+06, 2.696e+06+3000.0, 3000.0)
    ya = np.arange(-1.587e+06, 1.587e+06, 3000.0)
    
    x, y = projection.transform_point(longitude, latitude, ccrs.PlateCarree())

    print(longitude, latitude, x, y)
    
    xi = np.argmin(np.abs(x - xa))
    yi = np.argmin(np.abs(y - ya))
    
    return xi , yi
sst =  H.xarray(surface_name['SST'])
sst = sst.assign_coords(longitude=(((sst.longitude + 180) % 360) - 180))

xi_buoy, yi_buoy = get_nearest_point(projection, buoy_lon,  buoy_lat)
print(xi_buoy, yi_buoy)

xi, yi = get_nearest_point(projection, lon_center,  lat_center )
xslice = slice(xi - width, xi + width)
yslice = slice(yi - width,  yi + width)
print(sst.longitude.values[yi, xi], sst.latitude.values[yi, xi])

# Beging processing full HRRR subdomains in to new output xarray Dataset

In [ ]:

# First prepare the SST
sst = H.xarray(surface_name['SST'])
sst = sst.assign_coords(longitude=(((sst.longitude + 180) % 360) - 180))
sst = sst.rename({"longitude":"XLONG"})
sst = sst.rename({"latitude":"XLAT"})
sst = sst.rename_vars({"t":"SST"})

#print('Subset')
sst = sst.SST.isel(x=xslice, y=yslice)

#print('write')
sst.to_netcdf(out_file)

sst.close()
sst = None
#Release resources





In [ ]:
# First prepare the SST

t2m = H.xarray(surface_name['T2m'])
t2m = t2m.assign_coords(longitude=(((t2m.longitude + 180) % 360) - 180))
t2m = t2m.rename({"longitude":"XLONG"})
t2m = t2m.rename({"latitude":"XLAT"})
t2m = t2m.rename_vars({"t2m":"T2m"})

#print('Subset')
t2m = t2m.T2m.isel(x=xslice, y=yslice)

#print('write')
t2m.to_netcdf(out_file, 'a')

t2m.close()
t2m = None
#Release resources

In [ ]:
qv2m = H.xarray(surface_name['QV2m'])
qv2m = qv2m.assign_coords(longitude=(((qv2m.longitude + 180) % 360) - 180))
qv2m = qv2m.rename({"longitude":"XLONG"})
qv2m = qv2m.rename({"latitude":"XLAT"})
qv2m = qv2m.rename_vars({"sh2":"QV2m"})

#print('Subset')
qv2m = qv2m.QV2m.isel(x=xslice, y=yslice)

#print('write')
qv2m.to_netcdf(out_file, 'a')

qv2m.close()
qv2m = None

In [ ]:
u10m = H.xarray(surface_name['U10'])
u10m = u10m.assign_coords(longitude=(((u10m.longitude + 180) % 360) - 180))
u10m = u10m.rename({"longitude":"XLONG"})
u10m = u10m.rename({"latitude":"XLAT"})
u10m = u10m.rename_vars({"u10":"U10m"})
u10m = u10m.U10m.isel(x=xslice, y=yslice)

v10m = H.xarray(surface_name['V10'])
v10m = v10m.assign_coords(longitude=(((v10m.longitude + 180) % 360) - 180))
v10m = v10m.rename({"longitude":"XLONG"})
v10m = v10m.rename({"latitude":"XLAT"})
v10m = v10m.rename_vars({"v10":"V10m"})
v10m = v10m.V10m.isel(x=xslice, y=yslice)


lon = v10m.XLONG.values
lat = u10m.XLAT.values

for ti in tqdm.tqdm(range(u10m.values.shape[0])):
    u10m.values[ti, :,:],v10m.values[ti, :,:] = projection_pinacles.transform_vectors(projection,lon, lat,u10m.values[ti,:,:],v10m.values[ti,:,:])

#print('write')
u10m.to_netcdf(out_file, 'a')
v10m.to_netcdf(out_file, 'a')
u10m.close()
u10m = None

v10m.close()
v10m = None

In [ ]:
# Now prepare the surface pressure
pres = H.xarray(surface_name['PSFC'])
pres = pres.assign_coords(longitude=(((pres.longitude + 180) % 360) - 180))
pres = pres.rename({"longitude":"XLONG"})
pres = pres.rename({"latitude":"XLAT"})
pres = pres.rename_vars({"sp":"PSFC"})

pres = pres.PSFC.isel(x=xslice, y=yslice)
pres.to_netcdf(out_file, 'a')


#out_ds['PSFC'] =  pres.PSFC.isel(x=xslice, y=yslice, drop=True)
pres.close()  
pres = None

In [ ]:
# Now prepare the pressure levels
print('Read')
p =  Hnat.xarray(level_name['P']) 
p = p.assign_coords(longitude=(((p.longitude + 180) % 360) - 180))
p = p.rename({"longitude":"LONG"})
p = p.rename({"latitude":"LAT"})
p = p.rename_vars({"pres":"P"})
print('Subset')
p = p.P.isel(x=xslice, y=yslice)

print('Write')
p.to_netcdf(out_file, 'a')
p.close()
p = None



In [ ]:
# Now prepare the pressure levels
t =  Hnat.xarray(level_name['T']) 
t = t.assign_coords(longitude=(((t.longitude + 180) % 360) - 180))
t = t.rename({"longitude":"LONG"})
t = t.rename({"latitude":"LAT"})
t = t.rename_vars({"t":"T"})
t =  t.T.isel(x=xslice, y=yslice)
t.to_netcdf(out_file, 'a')
t.close()
t = None 

In [ ]:
# Now prepare the pressure levels
z =  Hnat.xarray(level_name['Z']) 
z = z.assign_coords(longitude=(((z.longitude + 180) % 360) - 180))
z = z.rename({"longitude":"LONG"})
z = z.rename({"latitude":"LAT"})
z = z.rename_vars({"gh":"Z"})
z =  z.Z.isel(x=xslice, y=yslice)
z.to_netcdf(out_file, 'a')
z.close()
z = None

In [ ]:
qv =  Hnat.xarray(level_name['QV'])
qv = qv.assign_coords(longitude=(((qv.longitude + 180) % 360) - 180))
qv = qv.rename({"longitude":"LONG"})
qv = qv.rename({"latitude":"LAT"})
qv = qv.rename_vars({"q":"QV"})
qv  =  qv.QV.isel(x=xslice, y=yslice)
qv.to_netcdf(out_file, 'a')
qv.close()
qv = None

In [ ]:
# Prepare the water vapor mixing ratio
qc =  Hnat.xarray(level_name['QC'])
qc = qc.assign_coords(longitude=(((qc.longitude + 180) % 360) - 180))
qc = qc.rename({"longitude":"LONG"})
qc = qc.rename({"latitude":"LAT"})
qc = qc.rename_vars({"clwmr":"QC"})
qc = qc.QC.isel(x=xslice, y=yslice)
qc.to_netcdf(out_file, 'a')
qc.close()
qc = None

In [ ]:
qi =  Hnat.xarray(level_name['QI'])
qi = qi.assign_coords(longitude=(((qi.longitude + 180) % 360) - 180))
qi = qi.rename({"longitude":"LONG"})
qi = qi.rename({"latitude":"LAT"})
qi = qi.rename_vars({"unknown":"QI"})
qi =  qi.QI.isel(x=xslice, y=yslice)
qi.to_netcdf(out_file, 'a')
qi.close()
qi = None

In [ ]:
u =  Hnat.xarray(level_name['U'])
v =  Hnat.xarray(level_name['V'])

u = u.assign_coords(longitude=(((u.longitude + 180) % 360) - 180))
v = v.assign_coords(longitude=(((v.longitude + 180) % 360) - 180))

u = u.rename({"longitude":"LONG_U"})
v = v.rename({"longitude":"LONG_V"})

u = u.rename({"latitude":"LAT_U"})
v = v.rename({"latitude":"LAT_V"})

u = u.rename_vars({"u":"U"})
v = v.rename_vars({"v":"V"})

u =  u.U.isel(x=xslice, y=yslice)
v =  v.V.isel(x=xslice, y=yslice)



#Rotate the winds 
lon = u.LONG_U.values
lat = u.LAT_U.values
for ti in tqdm.tqdm(range(u.values.shape[0])):
    for ki in range(u.values.shape[1]):
        u.values[ti, ki,:,:],v.values[ti, ki,:,:] = projection_pinacles.transform_vectors(projection,lon, lat,u.values[ti, ki,:,:],v.values[ti, ki,:,:])
        print(ki, np.amax(u.values[ti, ki,:,:]), np.amin(u.values[ti, ki,:,:]), np.amax(v.values[ti, ki,:,:]), np.amin(v.values[ti, ki,:,:]))


u.to_netcdf(out_file, 'a')
u.close()
u = None 
v.to_netcdf(out_file, 'a')
v.close()
v = None





In [ ]:
#print(out_ds)

In [ ]:
count = 0 
for h in tqdm.tqdm(surface_name):
    d = H.xarray(surface_name[h]).isel(x = xi_buoy,y = yi_buoy)
    
    if count == 0:
        d.to_netcdf('hrrr_ts.nc')
    else:
        d.to_netcdf('hrrr_ts.nc', 'a') 
    count += 1 
    

In [ ]:
import metpy 
